<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/unified_stack_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
# unified_stack.py — Metasystem + Dreamstack with real physics frames (single-file, notebook-safe)

from __future__ import annotations

import argparse
import csv
import json
import logging
import math
import random
import re
import shutil
import sys
import time
from dataclasses import dataclass, asdict, field
from pathlib import Path
from typing import Any, Dict, Iterable, List, Optional, Sequence, Tuple

# Hard deps for physics + images
import numpy as np
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

# Optional: networkx for causal graph (metasystem)
try:
    import networkx as nx
except Exception:
    nx = None


# ======================
# Notebook-safe argv
# ======================
def sanitize_argv(argv: Sequence[str]) -> List[str]:
    cleaned: List[str] = []
    skip_next = False
    for a in argv:
        if skip_next:
            skip_next = False
            continue
        if a == "-f":
            skip_next = True
            continue
        if a.endswith(".json") and "kernel" in a:
            continue
        if "ipykernel" in a or "jupyter" in a:
            continue
        cleaned.append(a)
    return cleaned

def setup_logging(verbosity: int = 0) -> None:
    lvl = logging.WARNING if verbosity <= 0 else logging.INFO if verbosity == 1 else logging.DEBUG
    logging.basicConfig(level=lvl, format="%(levelname)s | %(message)s")


def ensure_dir(p: Path) -> None:
    p.mkdir(parents=True, exist_ok=True)

def write_json(path: Path, obj: Any) -> None:
    ensure_dir(path.parent)
    path.write_text(json.dumps(obj, indent=2), encoding="utf-8")

def write_csv(path: Path, headers: Sequence[str], rows: Iterable[Sequence[Any]]) -> None:
    ensure_dir(path.parent)
    with path.open("w", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        w.writerow(list(headers))
        for r in rows:
            w.writerow(list(r))


# ======================
# Metasystem foundations
# ======================
@dataclass
class Reality:
    space: str = "3D Euclidean"
    time: str = "linear"
    objects: List[str] = field(default_factory=lambda: ["wave","particle","observer"])
    logic: str = "quantum-decoherent"
    ethics: str = "CEAS level-3"
    def mutate(self, *, logic: Optional[str]=None, add_objects: Optional[List[str]]=None, ethics: Optional[str]=None):
        if logic: self.logic = logic
        if add_objects:
            for o in add_objects:
                if o not in self.objects:
                    self.objects.append(o)
        if ethics: self.ethics = ethics
    def validate(self) -> List[str]:
        issues = []
        if self.time not in ("linear","cyclic","branching"):
            issues.append("unsupported time form")
        if self.logic not in ("quantum-decoherent","paraconsistent","hybrid_consistent"):
            issues.append("unsupported logic")
        if not self.ethics.startswith("CEAS level-"):
            issues.append("ethics not CEAS-tiered")
        return issues

@dataclass
class ESystem:
    A: Any
    L: Any
    R: Dict[str, Any]
    D: Any
    Sigma: Dict[str, Any]

def MetaTransform(E: ESystem) -> ESystem:
    R2 = dict(E.R)
    if R2.get("logic") == "quantum-decoherent":
        R2["logic"] = "hybrid_consistent"
    Sigma2 = dict(E.Sigma); Sigma2["version"] = Sigma2.get("version", 0) + 1
    return ESystem(A=E.A, L=E.L, R=R2, D=E.D, Sigma=Sigma2)

@dataclass
class HSystem:
    I: Any
    C: Dict[str, Any]
    E: ESystem
    Omega: Dict[str, Any]

def Psi(H: HSystem, Sigma_t: Dict[str, Any]) -> HSystem:
    C2 = dict(H.C); C2["policy_tag"] = Sigma_t.get("policy","default")
    return HSystem(I=H.I, C=C2, E=H.E, Omega={"audit":True,"Σ":Sigma_t})

@dataclass
class SSystem:
    O: Dict[str, Any]
    L: Any
    R: Dict[str, Any]
    E: ESystem

def Phi(S: SSystem, Psi_t: HSystem) -> SSystem:
    O2 = dict(S.O); O2["tick"] = O2.get("tick", 0) + 1
    return SSystem(O=O2, L=Psi_t.C.get("policy_tag","default"), R=S.R, E=S.E)


# ======================
# Symbolic engine
# ======================
SYMBOL_TABLE = {
    '∴✶': 'sacred',
    'Æ': 'divine_agent',
    'Δ⇌': 'translate_ontology',
    '⊘∨': 'conditional_divinity',
    '⊕†': 'atonement_merge',
    '♲⊚': 'recursion_loop'
}
TOKEN_PATTERN = r'([∴✶ÆΔ⇌⊘∨⊕†♲⊚\(\),])|([A-Za-z_][A-Za-z0-9_\-]*)|([^\s])'

def tokenize(text: str) -> List[str]:
    ts = re.findall(TOKEN_PATTERN, text)
    return [a or b or c for a,b,c in ts if (a or b or c)]

def parse(tokens: List[str]) -> Dict[str, Any]:
    expr: Dict[str, Any] = {}
    i = 0
    while i < len(tokens):
        t = tokens[i]
        if t in SYMBOL_TABLE:
            expr["operation"] = SYMBOL_TABLE[t]
            if i+1 < len(tokens) and tokens[i+1] == '(':
                i += 2
                args, cur, depth = [], [], 1
                while i < len(tokens) and depth > 0:
                    tok = tokens[i]
                    if tok == '(':
                        depth += 1; cur.append(tok)
                    elif tok == ')':
                        depth -= 1
                        if depth == 0:
                            if cur: args.append(''.join(cur).strip()); break
                            else: break
                        cur.append(tok)
                    elif tok == ',' and depth == 1:
                        args.append(''.join(cur).strip()); cur=[]
                    else:
                        cur.append(tok)
                    i += 1
                expr["arguments"] = args
        i += 1
    return expr

def interpret(expr: Dict[str, Any]) -> str:
    op = expr.get("operation"); args = expr.get("arguments", [])
    if op == "sacred":
        subj = args[0] if args else "Unknown"
        return f"[Declaration] '{subj}' marked as sacred."
    if op == "atonement_merge":
        a,b = (args+["X","Y"])[:2]; return f"[Ethical Merge] Attempting reconciliation between '{a}' and '{b}'."
    if op == "translate_ontology":
        a,b = (args+["A","B"])[:2]; return f"[Translation] Initiating ontology shift: {a} ↔ {b}"
    if op == "conditional_divinity":
        subj = args[0] if args else "proposition"
        return f"[Conditional Action] Executing divine function if '{subj}' is ethically coherent."
    if op == "recursion_loop":
        return "[Recursion] Loop authorized."
    return "[Unknown Operation]"


# ======================
# Physics: 2D φ⁴ field
# ======================
@dataclass
class PhysConfig:
    nx: int = 128
    ny: int = 128
    dx: float = 1.0
    dt: float = 0.05
    c: float = 1.0
    lam: float = 1.0
    v: float = 1.0
    eta: float = 0.01
    init: str = "random"  # random|blob|bias
    random_amp: float = 0.05
    blob_sigma: float = 18.0
    bias_value: float = 0.9
    cmap: str = "seismic"

def laplacian_5pt(a: np.ndarray, dx: float) -> np.ndarray:
    return (np.roll(a,1,0)+np.roll(a,-1,0)+np.roll(a,1,1)+np.roll(a,-1,1)-4*a)/(dx*dx)

def dV_dphi(phi: np.ndarray, lam: float, v: float) -> np.ndarray:
    return lam * phi * (phi*phi - v*v)

def gradients_central(a: np.ndarray, dx: float):
    gx = (np.roll(a,-1,0) - np.roll(a,1,0))/(2*dx)
    gy = (np.roll(a,-1,1) - np.roll(a,1,1))/(2*dx)
    return gx, gy

def energy_density(phi: np.ndarray, phi_prev: np.ndarray, cfg: PhysConfig) -> np.ndarray:
    vel = (phi - phi_prev) / cfg.dt
    Ek = 0.5 * (vel*vel)
    gx, gy = gradients_central(phi, cfg.dx)
    Eg = 0.5 * (cfg.c*cfg.c) * (gx*gx + gy*gy)
    Ep = 0.25 * cfg.lam * (phi*phi - cfg.v*cfg.v)**2
    return Ek + Eg + Ep

def total_energy(phi: np.ndarray, phi_prev: np.ndarray, cfg: PhysConfig) -> float:
    return float(energy_density(phi, phi_prev, cfg).sum() * cfg.dx * cfg.dx)

def init_field(cfg: PhysConfig, rng: np.random.Generator, shape: Tuple[int,int]) -> np.ndarray:
    nx, ny = shape
    if cfg.init == "random":
        return cfg.random_amp * (rng.random((nx, ny)) - 0.5)
    elif cfg.init == "blob":
        i = np.arange(nx); j = np.arange(ny)
        ii, jj = np.meshgrid(i, j, indexing="ij")
        r2 = (ii - (nx-1)/2.0)**2 + (jj - (ny-1)/2.0)**2
        return np.exp(-r2/(2.0*cfg.blob_sigma*cfg.blob_sigma))
    elif cfg.init == "bias":
        return np.full((nx, ny), cfg.bias_value, dtype=np.float32)
    else:
        return cfg.random_amp * (rng.random((nx, ny)) - 0.5)

def simulate_phi4_frames(out_dir: Path, frames: int, phys: PhysConfig, seed: int, save_png: bool=True) -> Dict[str, Any]:
    log = logging.getLogger("physics")
    rng = np.random.default_rng(seed)
    nx, ny = phys.nx, phys.ny
    phi = init_field(phys, rng, (nx, ny)).astype(np.float32)
    phi_prev = phi.copy()

    # Stability hint
    s = phys.c * phys.dt / phys.dx
    if s > 1.0 / math.sqrt(2.0):
        log.warning("CFL advisory: c*dt/dx=%.3f > 1/sqrt(2). Consider smaller dt.", s)

    frame_dir = out_dir
    ensure_dir(frame_dir)

    energies = []
    for t in range(frames):
        lap = laplacian_5pt(phi, phys.dx)
        force = (phys.c*phys.c)*lap - dV_dphi(phi, phys.lam, phys.v)
        damp = phys.eta * phys.dt
        phi_new = (2.0 - damp) * phi - (1.0 - damp) * phi_prev + (phys.dt*phys.dt) * force

        # Small stochastic kick for diversity
        phi_new += 0.001 * rng.standard_normal(phi.shape).astype(np.float32)

        # rotate buffers
        phi_prev, phi = phi, phi_new

        E = total_energy(phi, phi_prev, phys)
        energies.append(E)

        if save_png:
            fig = plt.figure(figsize=(4,4), dpi=120)
            ax = fig.add_axes([0,0,1,1])
            im = ax.imshow(phi.T, cmap=phys.cmap, origin="lower", interpolation="bilinear")
            ax.set_axis_off()
            fig.savefig(frame_dir / f"frame_{t:04d}.png", bbox_inches="tight", pad_inches=0)
            plt.close(fig)

    # Fitness: low energy variance + structure variance
    arrE = np.array(energies, dtype=np.float64)
    fitness = float(-arrE.std() + 0.001*arrE.mean())
    return {"frames": frames, "fitness": fitness, "E_mean": float(arrE.mean()), "E_std": float(arrE.std())}


# ======================
# Feature extraction + k-means
# ======================
def radial_power_spectrum(img: np.ndarray, nbins: int=32) -> np.ndarray:
    # img: 2D float array
    F = np.fft.fftn(img)
    P = np.abs(F)**2
    h, w = P.shape
    y, x = np.indices((h, w))
    cy, cx = (h-1)/2.0, (w-1)/2.0
    r = np.sqrt((y-cy)**2 + (x-cx)**2)
    r /= (r.max() + 1e-9)
    bins = np.linspace(0, 1.0, nbins+1)
    feat = np.zeros(nbins, dtype=np.float64)
    for i in range(nbins):
        mask = (r >= bins[i]) & (r < bins[i+1])
        feat[i] = P[mask].mean() if np.any(mask) else 0.0
    feat = np.log1p(feat)
    feat /= (np.linalg.norm(feat) + 1e-12)
    return feat

def load_png_as_gray(path: Path) -> np.ndarray:
    img = matplotlib.image.imread(path)
    if img.ndim == 3:
        img = img[..., :3].mean(axis=-1)
    img = img.astype(np.float32)
    if img.max() > 1.0:
        img /= 255.0
    return img

def kmeans(X: np.ndarray, k: int, iters: int=10, seed: int=42) -> Tuple[np.ndarray, np.ndarray]:
    rng = np.random.default_rng(seed)
    n, d = X.shape
    k = max(1, min(k, n))
    idx = rng.choice(n, size=k, replace=False)
    C = X[idx].copy()
    labels = np.zeros(n, dtype=np.int32)
    for _ in range(iters):
        # assign
        dists = np.linalg.norm(X[:, None, :] - C[None, :, :], axis=2)
        labels = np.argmin(dists, axis=1)
        # update
        for j in range(k):
            mask = (labels == j)
            if mask.any():
                C[j] = X[mask].mean(axis=0)
    return C, labels


# ======================
# Dreamstack-ish pipeline (with real physics PNG frames)
# ======================
def ds_run(out_dir: Path, steps: int, survivors: int, frames_per: int, seed: Optional[int],
           phys_cfg: Optional[PhysConfig]=None) -> Dict[str, Any]:
    log = logging.getLogger("ds.run")
    ensure_dir(out_dir / "runs")

    # Metasystem: initial reality and transforms recorded with the run
    R = Reality()
    issues_before = R.validate()
    R.mutate(logic="paraconsistent", add_objects=["meta-observer"], ethics="CEAS level-4")
    issues_after = R.validate()

    E0 = ESystem(A="A0", L="L0", R=asdict(R), D="D0", Sigma={"policy":"hybrid_consistent","version":0})
    E1 = MetaTransform(E0)
    H0 = HSystem(I="I0", C={"controller":"Ψ0"}, E=E1, Omega={})
    H1 = Psi(H0, E1.Sigma)
    S0 = SSystem(O={"tick":0}, L="L0", R=asdict(R), E=E1)
    S1 = Phi(S0, H1)

    write_json(out_dir / "runs" / "metasystem.json", {
        "issues_before": issues_before, "issues_after": issues_after,
        "E0": asdict(E0), "E1": asdict(E1),
        "H0": {"I":H0.I,"C":H0.C,"E":asdict(H0.E),"Omega":H0.Omega},
        "H1": {"I":H1.I,"C":H1.C,"E":asdict(H1.E),"Omega":H1.Omega},
        "S0": {"O":S0.O,"L":S0.L,"R":S0.R},
        "S1": {"O":S1.O,"L":S1.L,"R":S1.R},
    })

    rng = random.Random(seed)
    phys = phys_cfg or PhysConfig()

    survivors_meta = []
    for s in range(survivors):
        sid = f"survivor_{s:03d}"
        sdir = out_dir / "runs" / sid / "frames"
        ensure_dir(sdir)

        # Slightly vary physics per survivor for diversity
        ph = PhysConfig(
            nx=phys.nx, ny=phys.ny, dx=phys.dx, dt=phys.dt,
            c=phys.c * (0.95 + 0.1*rng.random()),
            lam=phys.lam * (0.8 + 0.4*rng.random()),
            v=phys.v,
            eta=phys.eta,
            init=random.choice(["random","blob","bias"]),
            random_amp=phys.random_amp,
            blob_sigma=phys.blob_sigma,
            bias_value=phys.bias_value,
            cmap=phys.cmap
        )

        stats = simulate_phi4_frames(sdir, frames_per, ph, seed=rng.randrange(10**9), save_png=True)
        survivors_meta.append({"id": sid, "fitness": stats["fitness"], "frames": frames_per,
                               "E_mean": stats["E_mean"], "E_std": stats["E_std"]})

    write_json(out_dir / "runs" / "survivors.json", survivors_meta)
    log.info("Run: %d survivors × %d frames (%dx%d grid)", survivors, frames_per, phys.nx, phys.ny)
    return {"count": len(survivors_meta)}

def ds_visualize(out_dir: Path, top_n: int) -> Dict[str, Any]:
    meta_path = out_dir / "runs" / "survivors.json"
    data = json.loads(meta_path.read_text(encoding="utf-8"))
    data.sort(key=lambda x: x["fitness"], reverse=True)
    rows = [[i, d["id"], d["fitness"], d["frames"], d["E_mean"], d["E_std"]] for i, d in enumerate(data)]
    write_csv(out_dir / "visualize" / "summary.csv", ["rank","id","fitness","frames","E_mean","E_std"], rows)
    write_json(out_dir / "visualize" / "top.json", data[:max(0, top_n)])
    return {"summarized": len(rows), "top": min(top_n, len(rows))}

def ds_compress(out_dir: Path, k: int, scope: str, top_n: int, seed: Optional[int]) -> Dict[str, Any]:
    top = json.loads((out_dir / "visualize" / "top.json").read_text(encoding="utf-8"))
    considered = top[:max(0, top_n)] if scope == "local" else top

    # Collect features from first N frames of each considered survivor
    feats: List[np.ndarray] = []
    idx_map: List[Tuple[str, int]] = []
    for entry in considered:
        sid = entry["id"]
        fdir = out_dir / "runs" / sid / "frames"
        files = sorted(fdir.glob("frame_*.png"))[:16]  # sample up to 16 frames
        for i, fp in enumerate(files):
            img = load_png_as_gray(fp)
            feat = radial_power_spectrum(img, nbins=48)
            feats.append(feat)
            idx_map.append((sid, i))
    if not feats:
        write_json(out_dir / "compress" / "motifs.json", {"k": k, "counts": [], "centroids": [], "assignments": []})
        return {"k": k, "considered": 0}

    X = np.vstack(feats)
    C, labels = kmeans(X, k=k, iters=10, seed=seed or 42)

    counts = [int((labels == j).sum()) for j in range(C.shape[0])]
    obj = {
        "k": int(C.shape[0]),
        "counts": counts,
        "assignments": [{"sid": sid, "frame_idx": int(fi), "label": int(l)} for (sid, fi), l in zip(idx_map, labels)],
        "centroids": C.tolist(),
    }
    write_json(out_dir / "compress" / "motifs.json", obj)
    return {"k": int(C.shape[0]), "considered": len(considered)}

def ds_replay(out_dir: Path, top_n: int, copy_assets: bool) -> Dict[str, Any]:
    top = json.loads((out_dir / "visualize" / "top.json").read_text(encoding="utf-8"))
    sel = top[:max(0, top_n)]
    rdir = out_dir / "replay"; ensure_dir(rdir)
    total_png = 0
    for m in sel:
        sid = m["id"]
        src = out_dir / "runs" / sid / "frames"
        dst = rdir / sid; ensure_dir(dst)
        write_json(dst / "replay.json", {"id": sid, "source": str(src)})
        if copy_assets and src.exists():
            dframes = dst / "frames"; ensure_dir(dframes)
            for fp in sorted(src.glob("frame_*.png")):
                shutil.copy2(fp, dframes / fp.name)
                total_png += 1
    return {"replays": len(sel), "copied_png": total_png}

def ds_fuse(out_dir: Path, top_n: int, strategy: str) -> Dict[str, Any]:
    top = json.loads((out_dir / "visualize" / "top.json").read_text(encoding="utf-8"))
    motifs = json.loads((out_dir / "compress" / "motifs.json").read_text(encoding="utf-8"))
    sel = top[:max(0, top_n)]
    if strategy == "average":
        score_hint = float(np.mean([s["fitness"] for s in sel])) if sel else 0.0
    elif strategy == "best":
        score_hint = float(max([s["fitness"] for s in sel])) if sel else 0.0
    else:  # mix
        scores = [s["fitness"] for s in sel]
        score_hint = float(np.median(scores)) if scores else 0.0
    fused = {
        "strategy": strategy,
        "survivors": [m["id"] for m in sel],
        "motifs": motifs.get("motifs", []),
        "score_hint": score_hint,
        "created_at": time.time(),
    }
    write_json(out_dir / "fuse" / "fused_config.json", fused)
    return {"fused": len(sel)}

# ======================
# High-level pipeline (+ metasystem extras)
# ======================
def pipeline(out_dir: Path, steps: int, survivors: int, frames_per: int,
             top_n: int, k: int, scope: str, copy_assets: bool,
             strategy: str, seed: Optional[int]) -> int:
    log = logging.getLogger("pipeline")
    log.info("Pipeline start")
    ds_run(out_dir, steps, survivors, frames_per, seed)
    ds_visualize(out_dir, top_n)
    ds_compress(out_dir, k, scope, top_n, seed)
    ds_replay(out_dir, top_n, copy_assets)
    ds_fuse(out_dir, top_n, strategy)

    # Metasystem artifacts: causal graph, ethics gate, symbolic evals
    if nx is not None:
        G = nx.DiGraph()
        G.add_edges_from([("E1","E2"),("E2","E3"),("E1","E4")])
        G.nodes["E1"]["state"] = {"object":"observer","property":"init"}
        G.nodes["E2"]["state"] = {"object":"field","interaction":"gravity"}
        past = sorted(nx.ancestors(G, "E3"))
        write_json(out_dir / "metasystem" / "causal_past_E3.json", {"past": past})
    else:
        write_json(out_dir / "metasystem" / "causal_past_E3.json", {"past": [], "note": "networkx not installed"})

    gate = "Action Permitted under Δ⇌ logic"
    write_json(out_dir / "metasystem" / "ethics_gate.json", {"decision": gate})

    texts = ["⊘∨(paradox)", "⊕†(clarity, loss)", "∴✶(Emergence)", "Δ⇌(Paradox_Tolerance, Coherence)"]
    evals = []
    for t in texts:
        expr = parse(tokenize(t)); evals.append({"text": t, "result": interpret(expr)})
    write_json(out_dir / "metasystem" / "symbols.json", {"evaluations": evals})

    log.info("Pipeline complete")
    return 0


# ======================
# CLI
# ======================
def build_parser() -> argparse.ArgumentParser:
    p = argparse.ArgumentParser(
        prog="unified_stack",
        description="Metasystem + Dreamstack unified pipeline (notebook-safe)"
    )
    p.add_argument("-v", "--verbose", action="count", default=0,
                   help="Increase verbosity (-v, -vv)")

    sub = p.add_subparsers(dest="cmd")  # not required; main handles help on no cmd

    # run
    pr = sub.add_parser("run", help="Run dreamstack with real physics frames (PNG)")
    pr.add_argument("--out-dir", type=Path, default=Path("out_unified"), dest="out_dir")
    pr.add_argument("--steps", type=int, default=200)
    pr.add_argument("--survivors", type=int, default=5)
    pr.add_argument("--frames-per", type=int, default=32, dest="frames_per")
    pr.add_argument("--seed", type=int, default=None)
    pr.add_argument("--nx", type=int, default=128)
    pr.add_argument("--ny", type=int, default=128)
    pr.add_argument("--dt", type=float, default=0.05)
    pr.add_argument("--c", type=float, default=1.0)
    pr.add_argument("--lam", type=float, default=1.0)
    pr.add_argument("--v", type=float, default=1.0)
    pr.add_argument("--eta", type=float, default=0.01)
    pr.add_argument("--init", choices=["random","blob","bias"], default="random")
    pr.add_argument("--cmap", type=str, default="seismic")
    pr.set_defaults(func=lambda ns: ds_run(
        ns.out_dir, ns.steps, ns.survivors, ns.frames_per, ns.seed,
        PhysConfig(nx=ns.nx, ny=ns.ny, dt=ns.dt, c=ns.c, lam=ns.lam, v=ns.v, eta=ns.eta, init=ns.init, cmap=ns.cmap)
    ) or 0)

    # visualize
    pv = sub.add_parser("visualize", help="Summarize survivors")
    pv.add_argument("--out-dir", type=Path, default=Path("out_unified"), dest="out_dir")
    pv.add_argument("--top-n", type=int, default=3, dest="top_n")
    pv.set_defaults(func=lambda ns: ds_visualize(ns.out_dir, ns.top_n) or 0)

    # compress
    pc = sub.add_parser("compress", help="Compress frame PNGs into spectral motifs")
    pc.add_argument("--out-dir", type=Path, default=Path("out_unified"), dest="out_dir")
    pc.add_argument("--k", type=int, default=8)
    pc.add_argument("--scope", choices=("local","global"), default="local")
    pc.add_argument("--top-n", type=int, default=3, dest="top_n")
    pc.add_argument("--seed", type=int, default=None)
    pc.set_defaults(func=lambda ns: ds_compress(ns.out_dir, ns.k, ns.scope, ns.top_n, ns.seed) or 0)

    # replay
    prp = sub.add_parser("replay", help="Prepare replays (optionally copy PNGs)")
    prp.add_argument("--out-dir", type=Path, default=Path("out_unified"), dest="out_dir")
    prp.add_argument("--top-n", type=int, default=3, dest="top_n")
    prp.add_argument("--copy", action="store_true")
    prp.set_defaults(func=lambda ns: ds_replay(ns.out_dir, ns.top_n, ns.copy) or 0)

    # fuse
    pf = sub.add_parser("fuse", help="Fuse configs")
    pf.add_argument("--out-dir", type=Path, default=Path("out_unified"), dest="out_dir")
    pf.add_argument("--top-n", type=int, default=3, dest="top_n")
    pf.add_argument("--strategy", choices=("average","best","mix"), default="mix")
    pf.set_defaults(func=lambda ns: ds_fuse(ns.out_dir, ns.top_n, ns.strategy) or 0)

    # pipeline
    pp = sub.add_parser("pipeline", help="Run full pipeline + metasystem artifacts")
    pp.add_argument("--out-dir", type=Path, default=Path("out_unified"), dest="out_dir")
    pp.add_argument("--steps", type=int, default=200)
    pp.add_argument("--survivors", type=int, default=4)
    pp.add_argument("--frames-per", type=int, default=32, dest="frames_per")
    pp.add_argument("--top-n", type=int, default=2, dest="top_n")
    pp.add_argument("--k", type=int, default=6)
    pp.add_argument("--scope", choices=("local","global"), default="local")
    pp.add_argument("--copy", action="store_true")
    pp.add_argument("--strategy", choices=("average","best","mix"), default="mix")
    pp.add_argument("--seed", type=int, default=None)
    pp.set_defaults(func=lambda ns: pipeline(
        ns.out_dir, ns.steps, ns.survivors, ns.frames_per,
        ns.top_n, ns.k, ns.scope, ns.copy, ns.strategy, ns.seed
    ))

    return p

def main(argv: Optional[List[str]] = None, *, exit: bool = True) -> int:
    raw = sys.argv[1:] if argv is None else argv
    argv_clean = sanitize_argv(raw)
    parser = build_parser()

    # Graceful help on no subcommand
    if not argv_clean or (argv_clean and argv_clean[0].startswith("-") and len(argv_clean) <= 2):
        try:
            ns, _ = parser.parse_known_args(argv_clean)
            setup_logging(getattr(ns, "verbose", 0))
        except SystemExit:
            setup_logging(0)
        parser.print_help()
        if exit:
            sys.exit(0)
        return 0

    try:
        ns = parser.parse_args(argv_clean)
    except SystemExit as e:
        if exit:
            raise
        return int(getattr(e, "code", 2) or 0)

    setup_logging(getattr(ns, "verbose", 0))
    code = int(ns.func(ns))
    if exit:
        sys.exit(code)
    return code

def cli(command: str, /, **kwargs) -> int:
    args: List[str] = [str(command)]
    v = kwargs.pop("verbose", None)
    if isinstance(v, int) and v > 0:
        args.extend(["-" + "v"*min(v,3)])
    for k, val in kwargs.items():
        flag = "--" + k.replace("_","-")
        if isinstance(val, bool):
            if val:
                args.append(flag)
        elif isinstance(val, (list, tuple)):
            for item in val:
                args.extend([flag, str(item)])
        elif val is not None:
            args.extend([flag, str(val)])
    return main(args, exit=False)

if __name__ == "__main__":
    main()